In [2]:
import torch
from jacobian import jacobian

In [3]:
class CNNNet(torch.nn.Module):
    def __init__(self):
        super(CNNNet, self).__init__()
        self.cnn = torch.nn.Conv2d(1, 3, 5)
    
    def forward(self, x):
        print('x: {}'.format(x.shape))
        x = torch.nn.functional.relu(self.cnn(x))
        print('co: {}'.format(x.shape))
        return x

# jacobian

- cnnnet: x(10,1,5,5) -> y(10,3,1,1)
- dy/dx: jacobian(y, x) => (10, 3, 1, 1, 1, 5, 5)

In [6]:
cnet = CNNNet()
batch = 10
x = torch.randn(batch, 1, 5, 5)  # x (10, 1,5,5)
x.requires_grad = True
y = cnet(x)  # y (10,3,1,1)

x: torch.Size([10, 1, 5, 5])
co: torch.Size([10, 3, 1, 1])


# calc jacobian

In [7]:
pred = jacobian(y, x, create_graph=True)

# naive calc jacobian

In [8]:
y_ = y.reshape(batch, -1)
total_y = y_.shape[1]
grad_x = []
for batch_y in y_:
    for target in batch_y:
        target.backward(retain_graph=True)
        grad_x.append(x.grad.unsqueeze(dim=0).clone().numpy())
        x.grad.zero_()
grad_x = np.array(grad_x).reshape(*y.shape, *x.shape)
true_val = np.einsum('ijklimno->ijklmno', grad_x)  # only cnn

In [9]:
assert (pred.detach().numpy() == true_val).all()